In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd


In [ ]:

# start = "2022-01-01"
start = "2022-01-01"
end = "2026-01-29"

#AIQ ARGT 

# qqqm = yf.download('QQQM', start, end)
# brkb = yf.download('BRK-B', start, end)
# spmo = yf.download('SPMO', start, end)
# aiq = yf.download('AIQ', start, end)
# argt = yf.download('ARGT', start, end)

In [ ]:
qqqm.to_csv('hist/qqqm_raw.csv')
# brkb.to_csv('hist/brk_b_raw.csv', index=False)
# spmo.to_csv('hist/spmo_raw.csv', index=False)
# aiq.to_csv('hist/aiq_raw.csv', index=False)
# argt.to_csv('hist/argt_raw.csv', index=False)



In [ ]:
raw_data = qqqm
ticker = 'QQQM'

In [ ]:
px_flds = ['close', 'high', 'low', 'open']
df = raw_data.copy()
df.columns = df.columns.droplevel(level=1).str.lower()
df.columns = [c+'_px' if c in px_flds else c for c in df.columns]
df = df.reset_index().rename(columns={'Date':'date'}).reset_index(drop=True)

df.insert(loc=1, column='ticker', value=ticker)
print(df.columns)
display(df)
hist_fn = f'{ticker.lower().replace("-","_")}_hist.csv'
df.to_csv(hist_fn, index=False)


In [ ]:
import pandas as pd
# ticker = 'BRK-B'
# hist_fn = f'hist/{ticker.lower().replace("-","_")}_hist.csv'
df = pd.read_csv(hist_fn)
df['date'] = pd.to_datetime(df['date'])
display(df)

In [ ]:
df.info()
df.dtypes

- Momentum (RSI) shows general strength [Lead indicator]

#Trend indicators (SMA, MACD) helps to identify direction of the trend. [Lag indicator]
#SMA8 > SMA20 >> Up trend.
#Need to apply same time frame as leading indicator (1 hour, daily etc.)
#Eg. If RSA > 50 and SMA8 > SMA20 in the same (hourly/ daily) candle, buy. But 90% this will not happen and we won't make money,

#Index Long side strategy
#Time Frame: Daily
#RSI(7) - 7 periods
#SMA(8)
#SMA(20)
#MACD(12,26,9)
#If RSI(7) > 50 >> Buy [Momentum]
#Wait for x candles for either trend conditions to become true
#1. SMA(8) > SMA(20) -> Uptrenddata['Long_Entry'] = macd_window.rolling(lookahead_days, min_periods=1).max().astype(bool)

#2. MACD(12,26,9) -> Uptrend
### Exit condition 1:
If SMA or MACD cross up happens within x day, we will hold on to our position. 
If SMA cross up or MACD do not happen within x day, we exit.
### Exit Condition 2:
If SMA cross up or MACD happens within x days, 
then hold position til either SMA cross down or MACD cross down (MACD histogram > 0)
- 4-5 years historical data

In [ ]:
def sma(series: pd.Series, window:int) -> pd.Series:
    return series.rolling(window=window).mean()


In [ ]:
close_px = df['close_px']
display(close_px)

In [ ]:
FAST_SMA_PERIOD = 8
SLOW_SMA_PERIOD = 20
df[f'sma_fast'] = sma(close_px, FAST_SMA_PERIOD)
df[f'sma_slow'] = sma(close_px, SLOW_SMA_PERIOD)
display(df)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(df['date'], df['sma_fast'], label='sma fast', color='blue')
plt.plot(df['date'], df['sma_slow'], label='sma slow', color='grey')
plt.title(f"{ticker} SMA {FAST_SMA_PERIOD}/{SLOW_SMA_PERIOD}")
plt.legend()
plt.show()

In [ ]:
#EMA
def ema(series:pd.Series, span:int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()


In [ ]:
FAST_EMA_PERIOD = 12
SLOW_EMA_PERIOD = 26
df[f'ema_fast'] = ema(close_px, FAST_EMA_PERIOD)
df[f'ema_slow'] = sma(close_px, SLOW_EMA_PERIOD)

In [ ]:
display(df)

In [ ]:
#MACD
def macd(ema_fast, ema_slow, signal_period:int):
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal_period)
    histogram = macd_line - signal_line
    return pd.DataFrame({
        'macd':macd_line,
        'signal':signal_line,
        'histogram':histogram
    })

In [ ]:
MACD_SIGNAL_PERIOD = 9
macd_df = macd(df[f'ema_fast'], df[f'ema_slow'], MACD_SIGNAL_PERIOD)
display(macd_df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(12,6))
plt.plot(macd_df.index, macd_df['macd'], label='macd', color='blue')
plt.plot(macd_df.index, macd_df['signal'], label='signal', color='grey')
plt.bar(macd_df.index, macd_df['histogram'], label='histogram', color='lightblue')
plt.title(f"{ticker} MACD")
plt.legend()
plt.show()

In [ ]:
df[['macd', 'signal']] = macd(df[f'ema_fast'], df[f'ema_slow'], MACD_SIGNAL_PERIOD)[['macd','signal']]
display(df)

- When MACD line > signal line, histogram is positive
- Shows bullish momentum and indicates potential long signal

In [ ]:
#RSI
def rsi(series:pd.Series, period:int):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
#     print(rs)
    return 100 - (100/(1+rs))

In [ ]:
RSI_PERIOD = 14
df['rsi'] = rsi(df['close_px'], RSI_PERIOD)
display(df)

In [ ]:
print(df.dtypes)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,8), sharex=True)

ax1.plot(df['date'], df['close_px'], label=f'{ticker} Close', color='blue')
ax1.set_title(f'{ticker} Close Price')
ax1.legend()
ax1.grid(True)

ax2.plot(df['date'], df['rsi'], label='RSI', color='orange')
ax2.axhline(80, color='red', linestyle='--', label='Overbought (80)')
ax2.axhline(20, color='purple', linestyle='--', label='Oversold (20)')
ax2.set_title(f'RSI {RSI_PERIOD}')
ax2.legend()
ax2.grid(True)

ax2.xaxis.set_major_locator(mdates.MonthLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
display(df)

In [ ]:
df['sma_cross_long'] = df['sma_fast'] > df['sma_slow']
df['rsi_long_signal'] = df['rsi'] > 50
df['macd_cross_long'] = (df['macd'] > df['signal']) & (df['macd'].shift(1) <= df['signal'].shift(1))
# df['macd_cross_short'] = (df['macd'] < df['signal']) & (df['macd'].shift(1) >= df['signal'].shift(1))
display(df)

In [ ]:
#Looking forward for MACD entry from RSI pre-signal
lookahead_days = 5
df['sma_cross_long_entry'] = df['sma_cross_long'].rolling(lookahead_days, min_periods=1).max().astype(bool)
df['macd_cross_long_entry'] = df['macd_cross_long'].rolling(lookahead_days, min_periods=1).max().astype(bool)
# display(df[df['macd_cross_long_entry']==True])
df['long_entry'] = df['rsi_long_signal'] & (df['sma_cross_long_entry'] | df['macd_cross_long_entry'])
display(df)
print(df.columns)

In [ ]:
entry_df = df.copy()
entry_df = entry_df[entry_df['long_entry']==True]
display(entry_df[['date', 'ticker', 'close_px', 'rsi_long_signal', 'sma_cross_long', 'macd_cross_long', 'long_entry']])
# print(f'entry_dates:{entry_df["date"].unique().tolist()}')

## Exit Condition 2:
### If SMA cross up or MACD happens within x days, hold position til either SMA cross down or MACD cross down (MACD histogram > 0)

In [ ]:
df['long_exit'] = (df['sma_cross_long']==False) | (df['macd_cross_long']==False)
# df = df.drop('sma_cross_short',axis=1)
display(df)



In [ ]:
exit_df = df.copy()
exit_df = exit_df[exit_df['long_exit']==True]
display(exit_df[['date', 'ticker', 'close_px', 'rsi_long_signal', 'sma_cross_long', 'macd_cross_long', 'long_exit']])

In [ ]:
df['holding'] = 0
df['days_in_position'] = 0
df['trade'] = ''

# Step 3: track holding and days in position
in_position = False
days_in_position = 0
holding_period = 5
# display(df[df['long_entry']][['date', 'long_entry','long_exit','holding','days_in_position']])

for i in range(len(df)):
    # Enter if entry signal and currently flat
    if df['long_entry'].iat[i] and not in_position:
#         print(f'i:{i} Buy Trigger')
        days_in_position = 1
        in_position = True
        df['holding'].iat[i] = 1
        df['trade'].iat[i] = 'BUY'
    # Already holding
    elif in_position and df['long_exit'].iat[i] and days_in_position<holding_period:
#         print(f'i:{i} Sell Trigger')
        in_position = False
        days_in_position = 0
        df['holding'].iat[i] = 0
        df['trade'].iat[i] = 'SELL'
    elif in_position and days_in_position==holding_period:
#         print(f'i:{i} Sell Trigger')
        in_position = False
        days_in_position = 0
        df['holding'].iat[i] = 0
        df['trade'].iat[i] = 'SELL'
    elif in_position:
        days_in_position += 1
        df['holding'].iat[i] = 1
        df['trade'].iat[i] = 'HOLD'
    df['days_in_position'].iat[i] = days_in_position
#     print(f'i:{i}, in_position:{in_position}, days_in_position:{row["days_in_position"]}')
        
# display(df[['date', 'long_entry','long_exit','holding','days_in_position']])
display(df[df['days_in_position']>=1][['date', 'long_entry','long_exit','holding','days_in_position','trade']])


## Backtest Strategy
#### Market-On-Close Execution
- Execute trade at or as close as possible to the official closing price of a trading day
- Signal at today's close -> trade executed at today's close -> PnL starts tomorrow


In [ ]:
# Backtest strategy
initial_capital = 100000
allocation = 0.5 #50% of capital

#scale position by allocation (dollar-based position)
df['position'] = df['holding'] * (allocation * initial_capital)
df.loc[df['trade']=='BUY', 'position_shrs'] = df['position'] // df['close_px'] # Can use / for fractional shares
df['position_shrs'] = df['position_shrs'].replace(0, np.nan).ffill().fillna(0)
df.loc[df['trade']=='SELL', 'position_shrs'] = 0
display(df[['date', 'long_entry', 'long_exit', 'holding', 'position', 'position_shrs']])
display(df[df['trade']!=''][['date', 'long_entry', 'long_exit', 'holding', 'position', 'position_shrs','trade']])

In [ ]:
# import numpy as np
# # Backtest strategy
# initial_capital = 100000
# allocation = 0.5 #50% of capital


# df['position'] = 0
# # df.loc[df['long_entry'],'position'] = 1
# # Only enter if long_entry is True AND position is flat yesterday
# df.loc[(df['long_entry'] & (~df['long_entry'].shift(1).fillna(False))), 'position'] = 1
# # display(df[df['date'].isin(['2022-02-09', '2022-02-10', '2022-02-11'])][['date', 'long_entry', 'long_exit', 'position']])

# # Set to 0 if today is long_exit and yesterday's position > 0
# df.loc[df['long_exit'] & (df['position'].shift(1) > 0), 'position'] = 0
# df['position'] = df['position'].ffill().fillna(0)
# display(df[df['date'].isin(['2022-02-07', '2022-02-08', '2022-02-09'])][['date', 'long_entry', 'long_exit', 'position']])

# # df['position'] = df['position'].astype(float)
# # df['position'] = df['position'].replace(0, np.nan).ffill().fillna(0)

# #scale position by allocation (dollar-based position)
# df['position'] = df['position'] * (allocation * initial_capital)
# df['position_shrs'] = 0
# df.loc[df['long_entry'],'position_shrs'] = df['position'] // df['close_px'] # Can use / for fractional shares
# df['position_shrs'] = df['position_shrs'].replace(0, np.nan).ffill().fillna(0)
# df.loc[df['long_exit'],'position_shrs'] = 0

# display(df[['date', 'long_entry', 'long_exit', 'position', 'position_shrs']])
# # display(df[df['position']>0])



In [ ]:
display(df[(df['long_entry']==True) ]) #| (df['long_exit']==True)
# display(df[(df['position']==1)])

In [ ]:
#Transaction costs
cost_per_shr = 0.005 # USD per share. Sometimes its cost_per_trade
# df['position'] = df['position'].astype(int)
# df['trade'] = df['position'].diff().abs()
df['trade_shrs'] = 0
df.loc[df['trade']=='BUY', 'trade_shrs'] = df['position_shrs']
df['trade_cost'] = df['trade_shrs'] * cost_per_shr

# df.loc[df['long_entry']==True,'trade'] = df['position']
display(df[df['trade']!=''])
display(df)

In [ ]:
df['daily_ret_c2c'] = df['close_px'].pct_change()
df['cum_ret'] = (1 + df['daily_ret_c2c']).cumprod()

#position shift 1 as today's return was earned by yesterday's position (entered at yesterday's close)
#trade executed at close(t), cost incurred on close
df['daily_pnl'] = df['daily_ret_c2c'] * df['position'].shift(1) - df['trade_cost']
df['cum_pnl'] = initial_capital + df['daily_pnl'].cumsum()
df['cum_ret_pct'] = (df['cum_pnl'] / initial_capital - 1) * 100

display(df[['close_px', 'position', 'daily_ret_c2c', 'cum_ret', 'daily_pnl', 'cum_pnl', 'cum_ret_pct']])



In [ ]:
# Metrics
total_return = round((df['cum_ret'].iloc[-1] - 1)*100, 2)
print(f'total_return: {total_return}%')

sharpe = round((df['daily_ret_c2c'].mean() / df['daily_ret_c2c'].std() * np.sqrt(252)),2)
print(f'sharpe: {sharpe}%')

max_drawdown = round(((df['cum_ret'] / df['cum_ret'].cummax() - 1).min()), 2)
print(f'max_drawdown: {max_drawdown}%')

total_pnl = round(df['daily_pnl'].sum(), 2)
print(f'total_pnl: ${total_pnl}')

metrics_df = pd.DataFrame({
    'ticker': [ticker],
    'total_return': [total_return],
    'sharpe_ratio': [sharpe],
    'max_drawdown': [max_drawdown],
    'total_pnl': [total_pnl],
})
# print(f'\nmetrics_df: \n{metrics_df}')
# metrics_df.to_csv(f'metrics_{ticker.lower().replace("-","_")}', index=False)

In [ ]:
trades_df = df.copy()
trades_df['trade_id'] = (trades_df['trade']=='BUY').cumsum()
trades_df.loc[trades_df['trade']=='', 'trade_id'] = np.nan
display(trades_df.tail(10)[['date', 'long_entry', 'long_exit', 'position', 'position_shrs', 'trade', 'trade_shrs', 'trade_id']])
display(trades_df[trades_df['trade_id'].notna()].tail(10)[['date', 'long_entry', 'long_exit', 'position', 'position_shrs', 'trade', 'trade_shrs', 'trade_id']])
# display(trades_df[trades_df['trade_id'].isin([1,2,3,4])][['date', 'long_entry', 'long_exit', 'position', 'position_shrs', 'trade', 'trade_shrs', 'trade_id']])

# Count trades: entries/exits
entries = trades_df[trades_df['trade']=='BUY']['trade'].count()
exits = trades_df[trades_df['trade']=='SELL']['trade'].count()
print(f'entries: {entries}, exits: {exits}')

# % of profitable trades
trade_pnl = trades_df.groupby('trade_id')['daily_pnl'].sum().dropna().reset_index()
print(f'trade_pnl: \n{trade_pnl}')
profitable = len(trade_pnl[trade_pnl['daily_pnl']>0])
non_profitable = len(trade_pnl[trade_pnl['daily_pnl']<0])
total_trades = len(trade_pnl)
pct_profitable = round((profitable / total_trades * 100), 2)
print(f'Profitable Trades: {profitable}')
print(f'Non-Profitable Trades: {non_profitable}')
print(f'Total Trades: {total_trades}')
print(f'% of profitable trades: {pct_profitable}')

# Average P&L per trade
avg_pnl_per_trade = round(trade_pnl['daily_pnl'].mean(), 2)
print(f'Average PnL per Trade: ${avg_pnl_per_trade}')

# Win rate = #winning trades / total trades
trades_metrics_df = pd.DataFrame({
    'entries': [entries],
    'exits': [exits],
    'profitable_trades': [profitable],
    'non_profitable_trades': [non_profitable],
    'exits': [exits],
    'total_trades': [total_trades],
    'pct_profitable': [pct_profitable],
    'avg_pnl_per_trade': [avg_pnl_per_trade],
})

recent_trades = trades_df.tail(10)[['date', 'long_entry', 'long_exit', 'position', 'position_shrs', 'trade', 'trade_shrs', 'trade_id']]
print(f'\nrecent_trades:\n{recent_trades}')
print(f'\ntrades_metrics_df:\n{trades_metrics_df}')

In [ ]:

display(trades_df)
recent_trades.to_csv(f'metrics/recent_trades_{ticker.lower().replace("-","_")}.csv', index=False)
trades_df.to_csv(f'metrics/trades_{ticker.lower().replace("-","_")}.csv', index=False)

In [ ]:
metrics_df = pd.concat([metrics_df, trades_metrics_df], axis=1)
print(f'\nmetrics_df: \n{metrics_df}')
metrics_df.to_csv(f'metrics/metrics_{ticker.lower().replace("-","_")}.csv', index=False)



In [ ]:
# Cumulative PnL / Equity Curve

import matplotlib.ticker as mtick
plt.figure(figsize=(12,6))
plt.plot(df['date'], df['cum_pnl'], label='Equity Curve', color='blue')
plt.axhline(initial_capital, color='orange', linestyle='--', label='Initial Capital')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.title(f'{ticker} Strategy Equity Curve')
plt.grid(True)
plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
plt.legend()
plt.show()
plt.savefig(f'{ticker.lower().replace("-","_")}_equity_curve.png', dpi=300)


In [ ]:

# Visualize trades

# Plot price, SMA, RSI, MACD, and mark Long/Short entries and exits

# Refine strategy

# Adjust indicator parameters, lookahead days, or exit conditions

#Volatility/ Exposure
#Profit factor = gross profit/ gross loss

In [ ]:
# aiq.to_csv('metrics/metrics_aiq.csv', index=False)

In [ ]:
import glob
import os





f_dir = 'metrics/'
files = glob.glob(os.path.join(f_dir, 'metrics_*'))

dfs = [pd.read_csv(f) for f in files]
metrics_df = pd.concat(dfs, ignore_index=True)
metrics_df = metrics_df.style.format({'total_return': "${:,.2f}",
                 'max_drawdown': '{:.2%}',
                 'total_pnl': '${:,.2f}',
                 'pct_profitable': '{:}%', #win rate
                 'avg_pnl_per_trade': '${:,.2f}',
                })

# print(metrics_df.to_string(formatters={
#     'total_return': lambda x: f"${x:,.2f}",
#     'daily_pnl': lambda x: f"${x:,.2f}",
#     'total_pnl': lambda x: f"${x:,.2f}",
#     'pct_profitable': lambda x: f"{x:.2%}",
#     'avg_pnl_per_trade': lambda x: f"${x:,.2f}",
# }))

display(metrics_df)

#Sharpe ratio 1+ decent, 1.5 very good. 2+ excellent
#Max drawdown 20-40%
#Win rate 40-60%
#Avg pnl per trade > 0
#
